# Your Title Here

**Name(s)**: Luke Lin, Andrew Yin

**Website Link**: (your website link)

## Code

In [1]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'

### Framing the Problem

In [3]:
# Read the Excel file into a pandas DataFrame
outage = pd.read_excel("outage.xlsx", sheet_name="Masterdata")

# Drop informational rows
outage_cleaned = outage.drop(range(4)).dropna(axis=1, how='all')

# Set column names based on the first row
outage_cleaned.columns = outage_cleaned.iloc[0]

# Drop rows related to units and variables
outage_cleaned = outage_cleaned.drop([4, 5])
outage_cleaned = outage_cleaned.drop(columns="variables")

# Combine 'OUTAGE.START.DATE' and 'OUTAGE.START.TIME' into a new datetime column
outage_cleaned['OUTAGE.START'] = pd.to_datetime(outage_cleaned['OUTAGE.START.DATE']) + pd.to_timedelta(outage_cleaned['OUTAGE.START.TIME'].astype(str))

# Combine 'OUTAGE.RESTORATION.DATE' and 'OUTAGE.RESTORATION.TIME' into a new datetime column
outage_cleaned['OUTAGE.RESTORATION'] = pd.to_datetime(outage_cleaned['OUTAGE.RESTORATION.DATE']) + pd.to_timedelta(outage_cleaned['OUTAGE.RESTORATION.TIME'].astype(str))

# Drop the original date and time columns
outage_cleaned = outage_cleaned.drop(['OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE', 'OUTAGE.RESTORATION.TIME'], axis=1)

# Replace "NA" with NaN for missing values
outage_cleaned.replace("NA", np.nan, inplace=True)

#Essential Columns Analysis
essential_col = [
    "U.S._STATE", 
    'OUTAGE.START',
    "OUTAGE.RESTORATION", 
    "CUSTOMERS.AFFECTED", 
    "OUTAGE.DURATION", 
    "DEMAND.LOSS.MW", 
    "RES.PRICE", 
    "NERC.REGION", 
    "CLIMATE.REGION",
    "CAUSE.CATEGORY",
    "TOTAL.SALES"
]
# Display the cleaned DataFrame
# outage_cleaned = outage_cleaned[essential_col]
print(outage_cleaned.shape)


(1534, 54)


### Baseline Model

In [ ]:
# TODO

### Final Model

In [ ]:
# TODO

### Fairness Analysis

In [ ]:
# TODO